In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# username and password
username = 'USERNAME'
password = 'PASSWORD'

driver = webdriver.Chrome()

login_url = 'https://www.instagram.com/accounts/login/'
driver.get(login_url)

time.sleep(3)

username_field = driver.find_element(By.NAME, 'username')
password_field = driver.find_element(By.NAME, 'password')

username_field.send_keys(username)
password_field.send_keys(password)

password_field.send_keys(Keys.RETURN)

time.sleep(5)

profile_url = 'https://www.instagram.com/manchesterunited/'
driver.get(profile_url)

scroll_pause_time = 2  # scrolling speed
desired_post_count = 25
post_count = 0

while post_count < desired_post_count:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    
    post_elements = driver.find_elements(By.CLASS_NAME, '_aagw')
    post_count = len(post_elements)
    
    if post_count >= desired_post_count:
        break

In [33]:
instagram_account = profile_url.split('/')[3]

In [34]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

In [35]:
name = soup.find('span', {'class':'x1lliihq x1plvlek xryxfnj x1n2onr6 x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye xvs91rp x1s688f x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj'})
name = name.get_text().strip()
print(name)

Manchester United


In [36]:
followers = soup.find_all('span', {'class':'_ac2a'})
followers = followers[1].get_text().strip()
print(followers)

63M


In [37]:
image = soup.find_all('img', {'class': 'xpdipgo x972fbf xcfux6l x1qhh985 xm0m39n xk390pu x5yr21d xdj266r x11i5rnm xat24cr x1mh8g0r xl1xv1r xexx8yu x4uap5 x18d9i69 xkhd6sd x11njtxf xh8yej3'})
image = image[1].get('src')
print(image)

https://instagram.fidr6-1.fna.fbcdn.net/v/t51.2885-19/366427995_842647677462173_3717966822466956580_n.jpg?stp=dst-jpg_s320x320&_nc_ht=instagram.fidr6-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=m6cY8WOoygYAX8Ip0Cj&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfCB1XDe_z1B78yGmT0Xz1sXxmf5O_MzK9XdmrMTOhr3Gw&oe=6531A69C&_nc_sid=8b3546


In [38]:
bio = soup.find('h1', {'dir':'auto'})
bio = bio.get_text().strip()
print(bio)

Home of Manchester United 🔴


In [31]:
post_links = soup.find_all("a", href=True)
post_links = [link['href'] for link in post_links if '/p/' in link['href']]

post_number = 1 
for post_link in post_links[:25]:
    post_url = f"https://www.instagram.com{post_link}"
    driver.get(post_url)
    post_source = driver.page_source
    post_soup = BeautifulSoup(post_source, 'html.parser')
    
    caption = post_soup.find("h1", {'class' : "_aacl _aaco _aacu _aacx _aad7 _aade"})
    
    try:
        likes = post_soup.find("span", class_="html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs").get_text()
    except AttributeError:
        likes = "Likes information not available"
        
    post_details = post_soup.find("time")
    
    date = post_details["datetime"][:10] if post_details else "Date information not available"

    post_type = "Image" if not post_soup.find("video") else "Video"

    tagged_people = [a.text for a in post_soup.find_all("a", class_="notranslate") if a.text.startswith("@")]
    hashtags = [tag.text for tag in post_soup.find_all("a", href=True) if tag["href"].startswith("/explore/tags/")]

    
    print(f"Post Number: {post_number}")
    print(f"Post URL: {post_url}")
    print(f"Caption: {caption}")
    print(f"Likes: {likes}")
    print(f"Date: {date}")
    print(f"Post Type: {post_type}")
    print(f"Tagged People: {', '.join(tagged_people)}")
    print(f"Hashtags: {', '.join(hashtags)}")
    print("")
    
    post_data = {
        "Post Number": post_number,
        "Post URL": post_url,
        "Caption": caption,
        "Likes": likes,
        "Date": date,
        "Post Type": post_type,
        "Tagged People": tagged_people,
        "Hashtags": hashtags
    }
    
    post_data_list.append(post_data)
    post_number += 1

# Account and post details in a dictionary
account_data = {
    "Account Name": name,
    "Followers": followers,
    "Bio": bio,
    "Profile Image URL": image,
    "Posts": post_data_list
}

with open(f"{instagram_account}_details.json", "w", encoding="utf-8") as json_file:
    json.dump(account_data, json_file, ensure_ascii=False, indent=4)

Post Number: 1
Post URL: https://www.instagram.com/p/Cyar-AbN8Zj/
Caption: None
Likes: 15,479
Date: 2023-10-15
Post Type: Image
Tagged People: @1MaryEarps
Hashtags: #MUWomen, #MUFC, #ManUtd, #Earps, #rip, #Jimratcliffeout, #Glazersout🔰, #GlazersOut, #GLAZERDIE, #glazersout

Post Number: 2
Post URL: https://www.instagram.com/p/CyaeSduu90F/
Caption: None
Likes: 38,325
Date: 2023-10-15
Post Type: Image
Tagged People: @AndyCole09
Hashtags: #MUFC, #ManUtd, #AndyCole, #glazersout🔰, #GlazerIsFullOfShit, #glazersout, #glazerout, #glazersout, #glazersout🔰, #radcliffeout, #glazersout, #unfollowmanunited, #glazersout

Post Number: 3
Post URL: https://www.instagram.com/p/CyaUJLJN2o5/
Caption: None
Likes: 49,985
Date: 2023-10-15
Post Type: Video
Tagged People: 
Hashtags: #IsraelIsISIS, #glazersout, #glazersout, #glazersout🔰, #glazersout

Post Number: 4
Post URL: https://www.instagram.com/p/CyaT-uEs6BO/
Caption: None
Likes: 100,515
Date: 2023-10-15
Post Type: Image
Tagged People: @AltayBayindir
Hash